# TP1: Recommendation

*By Daniel Deutsch and Kevin Kuhl*

In [2]:
from movielens_utils import *

# 1) Presentation of the Model

### Question 1.1

The minidata argument on the load_movielens function (if True) reduces the data to a 100x200 dimension.

In [6]:
R, mask = load_movielens("./datasets/ml-100k/u.data")
print(f"The R matrix has size {R.shape}")

The R matrix has size (943, 1682)


### Question 1.2

In [7]:
print(f"There are {R.shape[0]} users and {R.shape[1]} films in the dataset. The total number of grades is {mask.sum()}")

There are 943 users and 1682 films in the dataset. The total number of grades is 100000


# 2) Find $P$ when $Q^0$ is Fixed 

### Question 2.1

To find the gradient of the function $g$ we need to obtain the derivative in relation to $P$:

<br>

\begin{aligned}
    \nabla g(P) \quad = \quad \frac{\partial g}{\partial P} \quad & = \quad \frac{\partial}{\partial P} \left(\frac{1}{2} ||1_K \circ (R - Q^0 P) ||_F^2 + \frac{\rho}{2}||Q^0||_F^2 + \frac{\rho}{2} ||P||_F^2\right) \\
    & = \quad \frac{\partial}{\partial P} \left(\frac{1}{2} ||1_K \circ (R - Q^0 P) ||_F^2\right) + \underbrace{\frac{\partial}{\partial P} \left(\frac{\rho}{2}||Q^0||_F^2\right)}_\text{= 0} + \frac{\partial}{\partial P} \left(\frac{\rho}{2} ||P||_F^2\right) \\
    & = \quad \left(-\frac{2}{2} \left(Q^0\right)^T [1_K \circ (R - Q^0 P)]\right) + \left(\frac{\rho}{2} 2 P\right) \\
    & = \quad -\left(Q^0\right)^T [1_K \circ (R - Q^0 P)] + \rho P
\end{aligned}

### Question 2.2

### Question 2.3

### Question 2.4

### Question 2.5

# 3) Resolution of the Full Problem

### Question 3.1

### Question 3.2

### Question 3.3